# Pricing a Forward Contract using Option Prices and Discount

In [1]:
using CSV
using DelimitedFiles, DataFrames
using Statistics
#import PyPlot as plt
#using Plots
using GLM
using ShiftedArrays
using PyCall
using PlotlyJS

WebIO._IJuliaInit()

In [2]:
JNJ         = DataFrame(CSV.File("DATA/JNJ.csv"))

,Date,Open,High,Low,Close,Adj Close,Volume
,Date,Float64,Float64,Float64,Float64,Float64,Int64
1,2017-03-21,128.38,128.45,127.14,127.25,111.545,7578600
2,2017-03-22,127.05,127.65,126.21,126.26,110.677,10272700
3,2017-03-23,126.1,127.0,125.66,125.9,110.361,8508100
4,2017-03-24,125.86,126.36,125.13,125.48,109.993,6490300
5,2017-03-27,125.16,126.14,125.15,125.8,110.274,5004300
6,2017-03-28,125.62,125.78,124.78,125.66,110.151,5287600
7,2017-03-29,125.05,125.33,124.36,124.92,109.502,3971800
8,2017-03-30,124.74,125.43,124.28,124.66,109.274,4711700
9,2017-03-31,124.11,124.83,124.03,124.55,109.178,5657600


In [3]:
p = PlotlyJS.plot(candlestick(x    =JNJ[!, "Date" ],
                              open =JNJ[!, "Open" ],
                              high =JNJ[!, "High" ],
                              low  =JNJ[!, "Low"  ],
                              close=JNJ[!, "Close"] ),
                  Layout(title="JNJ stock Prices",
                         yaxis_title="JNJ Stock")
    )

data: [
  "candlestick with fields close, high, low, open, type, and x"
]

layout: "layout with fields margin, template, title, and yaxis"

In [4]:
CALL_1MONTH   = DataFrame(CSV.File("DATA/CALL_1MONTH.csv"  ))
CALL_3MONTHS  = DataFrame(CSV.File("DATA/CALL_3MONTHS.csv" ))
CALL_7MONTHS  = DataFrame(CSV.File("DATA/CALL_7MONTHS.csv" ))
CALL_10MONTHS = DataFrame(CSV.File("DATA/CALL_10MONTHS.csv"))
CALL_15MONTHS = DataFrame(CSV.File("DATA/CALL_15MONTHS.csv"))
CALL_22MONTHS = DataFrame(CSV.File("DATA/CALL_22MONTHS.csv"))

PUT_1MONTH    = DataFrame(CSV.File("DATA/PUT_1MONTH.csv"   ))
PUT_3MONTHS   = DataFrame(CSV.File("DATA/PUT_3MONTHS.csv"  ))
PUT_7MONTHS   = DataFrame(CSV.File("DATA/PUT_7MONTHS.csv"  ))
PUT_10MONTHS  = DataFrame(CSV.File("DATA/PUT_10MONTHS.csv" ))
PUT_15MONTHS  = DataFrame(CSV.File("DATA/PUT_15MONTHS.csv" ))
PUT_22MONTHS  = DataFrame(CSV.File("DATA/PUT_22MONTHS.csv" ))

,Contract Name,Last Trade Date,Strike,Last Price,Bid,Ask
,String,String,Float64,Float64,Float64,Float64
1,JNJ240119P00080000,2022-03-15 9:32AM EDT,80.0,1.25,0.95,2.02
2,JNJ240119P00085000,2022-03-07 3:51PM EDT,85.0,1.6,1.2,1.95
3,JNJ240119P00090000,2022-03-04 2:54PM EDT,90.0,1.98,1.5,2.52
4,JNJ240119P00095000,2022-03-10 2:06PM EDT,95.0,2.29,1.5,2.51
5,JNJ240119P00100000,2022-03-21 12:22PM EDT,100.0,1.7,1.46,2.34
6,JNJ240119P00105000,2022-03-17 3:29PM EDT,105.0,2.5,1.91,2.57
7,JNJ240119P00110000,2022-03-16 10:29AM EDT,110.0,2.91,1.96,3.1
8,JNJ240119P00115000,2022-03-21 10:46AM EDT,115.0,3.3,2.26,3.45
9,JNJ240119P00120000,2022-03-15 10:48AM EDT,120.0,3.71,3.25,4.05


In [5]:
#findall(in(CALL_1MONTH.Strike),PUT_1MONTH.Strike)

In [6]:
M1  = intersect(CALL_1MONTH.Strike,   PUT_1MONTH.Strike  )
M3  = intersect(CALL_3MONTHS.Strike,  PUT_3MONTHS.Strike )
M7  = intersect(CALL_7MONTHS.Strike,  PUT_7MONTHS.Strike )
M10 = intersect(CALL_10MONTHS.Strike, PUT_10MONTHS.Strike)
M15 = intersect(CALL_15MONTHS.Strike, PUT_15MONTHS.Strike)
M22 = intersect(CALL_22MONTHS.Strike, PUT_22MONTHS.Strike)
MA  = intersect(CALL_1MONTH.Strike,   PUT_1MONTH.Strike, 
                CALL_3MONTHS.Strike,  PUT_3MONTHS.Strike,
                CALL_7MONTHS.Strike,  PUT_7MONTHS.Strike,
                CALL_10MONTHS.Strike, PUT_10MONTHS.Strike,
                CALL_15MONTHS.Strike, PUT_15MONTHS.Strike,
                CALL_22MONTHS.Strike, PUT_22MONTHS.Strike)
println(M22[1], "\t",M22[length(M22)])
println(MA[ 1], "\t",MA[ length(MA )])

80.0	250.0
150.0	175.0


In [7]:
INTEREST_RATES = DataFrame(CSV.File("DATA/INTEREST_RATES.csv"))

,USD,03-18-2022,03-17-2022,03-16-2022,03-15-2022,03-14-2022
,String,String,String,String,String,String
1,USD LIBOR - overnight,0.32871 %,0.32600 %,0.08029 %,0.07957 %,0.07914 %
2,USD LIBOR - 1 month,0.44657 %,0.44857 %,0.46757 %,0.44143 %,0.43057 %
3,USD LIBOR - 3 months,0.93400 %,0.92786 %,0.94814 %,0.91643 %,0.88471 %
4,USD LIBOR - 6 months,1.28757 %,1.27443 %,1.25671 %,1.23786 %,1.17500 %
5,USD LIBOR - 12 months,1.78643 %,1.77571 %,1.71529 %,1.68514 %,1.65686 %


In [8]:
INTEREST_RATES = Dict("overnight"=>.0032871,
                      "1 month"  =>.0044857,
                      "3 month"  =>.0093400,
                      "6 month"  =>.0128757,
                      "12 month" =>.0178643)
S0 = JNJ.Close[length(JNJ.Close)]
Maturity1  = 1  / 12
Maturity3  = 3  / 12
Maturity7  = 7  / 12
Maturity10 = 10 / 12
Maturity15 = 15 / 12
Maturity22 = 22 / 12
Maturities = [1,3,7,10,15,22]

6-element Vector{Int64}:
  1
  3
  7
 10
 15
 22

In [38]:
function Discount_Factor(Call1, Call2, Put1, Put2, Strike1, Strike2)
    "Compute the discount factor using Box Spread"
    return (   Call1 -   Call2 + 
                Put2 -    Put1 ) /
           ( Strike2 - Strike1 )
end

function Dividend(S0, Maturity, Interest_rate, Strike, Call, Put, Discount)
    "Compute the dividend through the forward contract formula obtained using the put-call parity"
    return S0 * ( 1 + Interest_rate * Maturity ) + ( Put - Call ) / Discount - Strike
end

function Dividend_Discount(S0, Maturity, Strike, Call, Put, Discount)
    "Compute the dividend through the forward contract formula obtained using the put-call parity"
    return S0 / Discount + ( Put - Call ) / Discount - Strike
end

Dividend_Discount (generic function with 1 method)

In [36]:
Discount1 = Discount_Factor(CALL_1MONTH[CALL_1MONTH.Strike.==MA[         1],"Last Price"][1],
                            CALL_1MONTH[CALL_1MONTH.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_1MONTH[  PUT_1MONTH.Strike.==MA[         1],"Last Price"][1],
                            PUT_1MONTH[  PUT_1MONTH.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discount3 = Discount_Factor(CALL_3MONTHS[CALL_3MONTHS.Strike.==MA[         1],"Last Price"][1],
                            CALL_3MONTHS[CALL_3MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_3MONTHS[  PUT_3MONTHS.Strike.==MA[         1],"Last Price"][1],
                            PUT_3MONTHS[  PUT_3MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discount7 = Discount_Factor(CALL_7MONTHS[CALL_7MONTHS.Strike.==MA[         1],"Last Price"][1],
                            CALL_7MONTHS[CALL_7MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_7MONTHS[  PUT_7MONTHS.Strike.==MA[         1],"Last Price"][1],
                            PUT_7MONTHS[  PUT_7MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discount10 = Discount_Factor(CALL_10MONTHS[CALL_10MONTHS.Strike.==MA[         1],"Last Price"][1],
                            CALL_10MONTHS[CALL_10MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_10MONTHS[  PUT_10MONTHS.Strike.==MA[         1],"Last Price"][1],
                            PUT_10MONTHS[  PUT_10MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discount15 = Discount_Factor(CALL_15MONTHS[CALL_15MONTHS.Strike.==MA[         1],"Last Price"][1],
                            CALL_15MONTHS[CALL_15MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_15MONTHS[  PUT_15MONTHS.Strike.==MA[         1],"Last Price"][1],
                            PUT_15MONTHS[  PUT_15MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discount22 = Discount_Factor(CALL_22MONTHS[CALL_22MONTHS.Strike.==MA[         1],"Last Price"][1],
                            CALL_22MONTHS[CALL_22MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            PUT_22MONTHS[  PUT_22MONTHS.Strike.==MA[         1],"Last Price"][1],
                            PUT_22MONTHS[  PUT_22MONTHS.Strike.==MA[length(MA)],"Last Price"][1],
                            MA[1], MA[length(MA)])
Discounts = [Discount1, Discount3, Discount7, Discount10, Discount15, Discount22]

println((1 ./ Discounts .- 1) ./ (Maturities ./ 12))
inter = log.(1 ./ Discounts) ./ (Maturities ./ 12)
println(inter)

[-0.0952380952380949, 0.15110004151100043, 0.045761220299207796, 0.04947938359017083, -0.004771371769383847, -0.10199556541019957]
[-0.09561803579012214, 0.14831603296568302, 0.0451611005821746, 0.04848649756204674, -0.004785657342038121, -0.11291681966417791]


In [37]:
PlotlyJS.plot(Maturities, inter)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin and template"

In [40]:
Dividend1  = Dividend(S0, Maturity1, INTEREST_RATES["1 month"],  S0, 
                      CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_1MONTH[  PUT_1MONTH.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[1])
Dividend3  = Dividend(S0, Maturity3, INTEREST_RATES["3 month"],  S0, 
                      CALL_3MONTHS[CALL_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_3MONTHS[  PUT_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[2])
Dividend7  = Dividend(S0, Maturity7, INTEREST_RATES["6 month"],  S0, 
                      CALL_7MONTHS[CALL_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_7MONTHS[  PUT_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[3])
Dividend10 = Dividend(S0, Maturity10, INTEREST_RATES["12 month"],  S0, 
                     CALL_10MONTHS[CALL_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_10MONTHS[  PUT_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[4])
Dividend15 = Dividend(S0, Maturity15, INTEREST_RATES["12 month"],  S0, 
                     CALL_15MONTHS[CALL_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_15MONTHS[  PUT_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[5])
Dividend22 = Dividend(S0, Maturity22, INTEREST_RATES["12 month"],  S0, 
                     CALL_22MONTHS[CALL_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_22MONTHS[  PUT_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[6])
Dividends = [Dividend1,Dividend3,Dividend7,Dividend10,Dividend15,Dividend22]

6-element Vector{Float64}:
 0.13480109194921397
 0.2837183894146449
 1.8984082187061802
 3.060970891102073
 5.693507089197368
 9.91321446091382

In [41]:
Dividend1  = Dividend_Discount(S0, Maturity1,  S0, 
                      CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_1MONTH[  PUT_1MONTH.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[1])
Dividend3  = Dividend_Discount(S0, Maturity3,  S0, 
                      CALL_3MONTHS[CALL_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_3MONTHS[  PUT_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[2])
Dividend7  = Dividend_Discount(S0, Maturity7,  S0, 
                      CALL_7MONTHS[CALL_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_7MONTHS[  PUT_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts[3])
Dividend10 = Dividend_Discount(S0, Maturity10,  S0, 
                     CALL_10MONTHS[CALL_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_10MONTHS[  PUT_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[4])
Dividend15 = Dividend_Discount(S0, Maturity15,  S0, 
                     CALL_15MONTHS[CALL_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_15MONTHS[  PUT_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[5])
Dividend22 = Dividend_Discount(S0, Maturity22,  S0, 
                     CALL_22MONTHS[CALL_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_22MONTHS[  PUT_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts[6])

Dividends_Discount = [Dividend1,Dividend3,Dividend7,Dividend10,Dividend15,Dividend22]

6-element Vector{Float64}:
  -1.3181745714285569
   6.480049662100441
   5.252402357289498
   7.667288464806347
   0.746481137176886
 -28.506665918699184

In [12]:
Discount1  = Discount_Factor(CALL_1MONTH[CALL_1MONTH.Strike.==M1[         1],"Last Price"][1],
                             CALL_1MONTH[CALL_1MONTH.Strike.==M1[length(M1)],"Last Price"][1],
                             PUT_1MONTH[  PUT_1MONTH.Strike.==M1[         1],"Last Price"][1],
                             PUT_1MONTH[  PUT_1MONTH.Strike.==M1[length(M1)],"Last Price"][1],
                             M1[1], M1[length(M1)])
Discount3  = Discount_Factor(CALL_3MONTHS[CALL_3MONTHS.Strike.==M3[         1],"Last Price"][1],
                             CALL_3MONTHS[CALL_3MONTHS.Strike.==M3[length(M3)],"Last Price"][1],
                             PUT_3MONTHS[  PUT_3MONTHS.Strike.==M3[         1],"Last Price"][1],
                             PUT_3MONTHS[  PUT_3MONTHS.Strike.==M3[length(M3)],"Last Price"][1],
                             M3[1], M3[length(M3)])
Discount7  = Discount_Factor(CALL_7MONTHS[CALL_7MONTHS.Strike.==M7[         1],"Last Price"][1],
                             CALL_7MONTHS[CALL_7MONTHS.Strike.==M7[length(M7)],"Last Price"][1],
                             PUT_7MONTHS[  PUT_7MONTHS.Strike.==M7[         1],"Last Price"][1],
                             PUT_7MONTHS[  PUT_7MONTHS.Strike.==M7[length(M7)],"Last Price"][1],
                             M7[1], M7[length(M7)])
Discount10 = Discount_Factor(CALL_10MONTHS[CALL_10MONTHS.Strike.==M10[         1 ],"Last Price"][1],
                             CALL_10MONTHS[CALL_10MONTHS.Strike.==M10[length(M10)],"Last Price"][1],
                             PUT_10MONTHS[  PUT_10MONTHS.Strike.==M10[         1 ],"Last Price"][1],
                             PUT_10MONTHS[  PUT_10MONTHS.Strike.==M10[length(M10)],"Last Price"][1],
                             M10[1], M10[length(M10)])
Discount15 = Discount_Factor(CALL_15MONTHS[CALL_15MONTHS.Strike.==M15[         1 ],"Last Price"][1],
                             CALL_15MONTHS[CALL_15MONTHS.Strike.==M15[length(M15)],"Last Price"][1],
                             PUT_15MONTHS[  PUT_15MONTHS.Strike.==M15[         1 ],"Last Price"][1],
                             PUT_15MONTHS[  PUT_15MONTHS.Strike.==M15[length(M15)],"Last Price"][1],
                             M15[1], M15[length(M15)])
Discount22 = Discount_Factor(CALL_22MONTHS[CALL_22MONTHS.Strike.==M22[         1 ],"Last Price"][1],
                             CALL_22MONTHS[CALL_22MONTHS.Strike.==M22[length(M22)],"Last Price"][1],
                             PUT_22MONTHS[  PUT_22MONTHS.Strike.==M22[         1 ],"Last Price"][1],
                             PUT_22MONTHS[  PUT_22MONTHS.Strike.==M22[length(M22)],"Last Price"][1],
                             M22[1], M22[length(M22)])
Discounts_VAR = [Discount1, Discount3, Discount7, Discount10, Discount15, Discount22]

6-element Vector{Float64}:
 1.0211428571428571
 0.9442631578947368
 0.9648421052631578
 1.0156315789473684
 1.0296111111111113
 0.9791764705882353

In [45]:
Dividend1  = Dividend(S0, Maturity1, INTEREST_RATES["1 month"],  S0, 
                      CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_1MONTH[  PUT_1MONTH.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[1])
Dividend3  = Dividend(S0, Maturity3, INTEREST_RATES["3 month"],  S0, 
                      CALL_3MONTHS[CALL_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_3MONTHS[  PUT_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[2])
Dividend7  = Dividend(S0, Maturity7, INTEREST_RATES["6 month"],  S0, 
                      CALL_7MONTHS[CALL_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_7MONTHS[  PUT_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[3])
Dividend10 = Dividend(S0, Maturity10, INTEREST_RATES["12 month"],  S0, 
                     CALL_10MONTHS[CALL_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_10MONTHS[  PUT_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[4])
Dividend15 = Dividend(S0, Maturity15, INTEREST_RATES["12 month"],  S0, 
                     CALL_15MONTHS[CALL_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_15MONTHS[  PUT_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[5])
Dividend22 = Dividend(S0, Maturity22, INTEREST_RATES["12 month"],  S0, 
                     CALL_22MONTHS[CALL_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_22MONTHS[  PUT_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[6])
Dividends_VAR = [Dividend1,Dividend3,Dividend7,Dividend10,Dividend15,Dividend22]

6-element Vector{Float64}:
  0.1339072910413961
  0.28116817344800893
  1.903962850792169
  3.0360564133481205
  5.6524755575648555
 10.985744398035848

In [46]:
Dividend1  = Dividend_Discount(S0, Maturity1, S0, 
                      CALL_1MONTH[CALL_1MONTH.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_1MONTH[  PUT_1MONTH.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[1])
Dividend3  = Dividend_Discount(S0, Maturity3, S0, 
                      CALL_3MONTHS[CALL_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_3MONTHS[  PUT_3MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[2])
Dividend7  = Dividend_Discount(S0, Maturity7, S0, 
                      CALL_7MONTHS[CALL_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                      PUT_7MONTHS[  PUT_7MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                      Discounts_VAR[3])
Dividend10 = Dividend_Discount(S0, Maturity10, S0, 
                     CALL_10MONTHS[CALL_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_10MONTHS[  PUT_10MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[4])
Dividend15 = Dividend_Discount(S0, Maturity15, S0, 
                     CALL_15MONTHS[CALL_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_15MONTHS[  PUT_15MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[5])
Dividend22 = Dividend_Discount(S0, Maturity22, S0, 
                     CALL_22MONTHS[CALL_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1],
                     PUT_22MONTHS[  PUT_22MONTHS.Strike.==Int(round(S0)),"Last Price"][1], 
                     Discounts_VAR[6])
Dividends_Discount_VAR = [Dividend1,Dividend3,Dividend7,Dividend10,Dividend15,Dividend22]

6-element Vector{Float64}:
 -3.5515276172355925
 10.193164024524833
  6.9617672555095
 -2.257733264859837
 -3.2800797425133794
  8.977733905082289

In [47]:
function PlotDiscouts(Maturities, Disc, Disc_VAR)
    Div = PlotlyJS.scatter(;x=Maturities, y=Disc,     mode="markers+lines", name="Var1" )
    Var = PlotlyJS.scatter(;x=Maturities, y=Disc_VAR, mode="markers+lines", name="Var2" )
    layout = Layout(title="Discount factor", yaxis_title="Discount factor", xaxis_title="Maturity [Months]")
    return PlotlyJS.plot([Div,Var], layout)
end

PlotDiscouts(Maturities, Discounts, Discounts_VAR)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [15]:
function PlotDividends(Maturities, Divid, Divid_VAR)
    Div = PlotlyJS.scatter(;x=Maturities, y=Divid,     mode="markers+lines", name="Var1" )
    Var = PlotlyJS.scatter(;x=Maturities, y=Divid_VAR, mode="markers+lines", name="Var2" )
    layout = Layout(title="Dividend expected", yaxis_title="Dividend payed", xaxis_title="Maturity [Months]")
    return PlotlyJS.plot([Div,Var], layout)
end

PlotDividends(Maturities, Dividends, Dividends_VAR)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [48]:
PlotDividends(Maturities, Dividends_Discount, Dividends_Discount_VAR)

data: [
  "scatter with fields mode, name, type, x, and y",
  "scatter with fields mode, name, type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"

In [16]:
Discount_arr10 = [Discount_Factor(CALL_10MONTHS[CALL_10MONTHS.Strike.==M10[          1+i],"Last Price"][1],
                                  CALL_10MONTHS[CALL_10MONTHS.Strike.==M10[length(M10)-i],"Last Price"][1],
                                  PUT_10MONTHS[  PUT_10MONTHS.Strike.==M10[          1+i],"Last Price"][1],
                                  PUT_10MONTHS[  PUT_10MONTHS.Strike.==M10[length(M10)-i],"Last Price"][1],
                                  M10[1+1], M10[length(M10)-1]) for i in 0:Int(round(length(M10)/2)-1)]

16-element Vector{Float64}:
 1.1026857142857143
 0.9695428571428573
 0.9341142857142857
 0.7937714285714286
 0.7234285714285715
 0.6436571428571428
 0.6048571428571429
 0.5544571428571429
 0.4742285714285715
 0.3593714285714285
 0.3309714285714286
 0.27285714285714285
 0.2290285714285714
 0.15342857142857144
 0.11548571428571427
 0.06405714285714285

In [17]:
sc = PlotlyJS.scatter(x=[M10[length(M10)-i]-M10[i+1] for i in 0:Int(round(length(M10)/2)-1)],y=Discount_arr10)
layout = Layout(title="Discount with respect to the strike difference", yaxis_title="Discount factor", xaxis_title="Strike difference [\$]")
    
PlotlyJS.plot(sc, layout)

data: [
  "scatter with fields type, x, and y"
]

layout: "layout with fields margin, template, title, xaxis, and yaxis"